In [1]:
from elisa.data.ogip import Data

In [2]:
det = 'n7'
spec = f'/Users/xuewc/ObsData/GRB231115A/GBM/{det}.pha'
back = f'/Users/xuewc/ObsData/GRB231115A/GBM/{det}.bak'
resp = f'/Users/xuewc/ObsData/GRB231115A/GBM/{det}.rsp'
d = Data((8, 900), spec, back, resp, name=det, group='bpos', scale=0.001)

In [4]:
import os
os.sys.path.append(os.path.abspath('../..'))
from elisa.src.elisa.data.ogip import Data
from elisa.v0.data import Data as Data0

import numpy as np

det = 'n7'
spec = f'/Users/xuewc/ObsData/GRB231115A/GBM/{det}.pha'
back = f'/Users/xuewc/ObsData/GRB231115A/GBM/{det}.bak'
resp = f'/Users/xuewc/ObsData/GRB231115A/GBM/{det}.rsp'
d = Data((8, 900), spec, back, resp, name=det, group='bpos', scale=0.001)
d0 = Data0((8, 900), spec, back, resp, name=det, group='bpos', scale=0.001)
path = '/Users/xuewc/ObsData/FRB221014/HXMT/'
d = Data([2, 10], f'{path}/LE_optbmin5.fits',
          f'{path}/LE_phabkg20s_g0_0-94.pha', f'{path}/LE_rsp.rsp',
          group='bmin', scale=25)
d0 = Data0([2, 10], f'{path}/LE_optbmin5.fits',
          f'{path}/LE_phabkg20s_g0_0-94.pha', f'{path}/LE_rsp.rsp',
          group='bmin', scale=25)
for i in ['back_counts', 'back_error', 'net_counts', 'net_error', 'net_spec', 'ch_emin', 'ch_emax', 'ch_width', 'ch_error', 'resp_matrix', 'ph_egrid']:
    if i == 'ch_error':
        print(i, np.abs(getattr(d, i)-np.abs(getattr(d0, i))).max())
    elif i == 'ph_egrid':
        print(i, np.abs(d.ph_egrid-d0.ph_ebins).max())
    else:
        print(i, np.abs(getattr(d, i)-getattr(d0, i)).max())

back_counts 0.0
back_error 0.0
net_counts 4.440892098500626e-16
net_error 4.440892098500626e-16
net_spec 1.7763568394002505e-15
ch_emin 0.0
ch_emax 0.0
ch_width 0.0
ch_error 4.963413395842053e-07
resp_matrix 0.0
ph_egrid 0.0


In [14]:
import os
os.sys.path.append(os.path.abspath('../..'))
import jax
jax.config.update("jax_enable_x64", True)
from elisa.src.elisa import model
from elisa.v0 import model as model0

import numpy as np

test = [
 'Band',
 'BandEp',
 'BlackBody',
 'BlackBodyRad',
 'Comptonized',
 'CutoffPowerlaw',
 'OTTB',
 'Powerlaw',
]

for mstr in test:
    print(mstr)
    m = getattr(model0, mstr)()
    mname = m.comps_name[0]
    params = getattr(m, m.comps_name[0])._config
    print(params)
    print()
    params = [v[0] for v in params.values()]

    eval1=m.eval(params, np.geomspace(1, 10, 100))
    
    if mstr == 'Powerlaw':
        kw = {}
    else:
        kw = dict(method='simpson')
    m2 = getattr(model, mstr)(**kw)
    mname = m2.__class__.__name__.lower()
    print(m2._default)
    print()
    params = {i[0]: i[2] for i in m2._default}

    eval2 = m2._wrapped_func(np.geomspace(1, 10, 100), {mname: params})

    if not np.allclose(eval1, eval2):
        print(np.abs((eval1-eval2)/eval1).max())
        print(mstr)

Band
{'alpha': [-1.0, -10.0, 5.0, False, False], 'beta': [-2.0, -10.0, 10.0, False, False], 'Ec': [300.0, 10.0, 10000.0, False, False], 'norm': [1, 1e-10, 10000000000.0, False, False]}

(('alpha', '\\alpha', -1.0, -10.0, 5.0, False, False), ('beta', '\\beta', -2.0, -10.0, 10.0, False, False), ('Ec', '\\E_{\\rm c}', 300.0, 10.0, 10000.0, False, False), ('K', 'K', 1, 1e-10, 10000000000.0, False, False))

BandEp
{'alpha': [-1.0, -10.0, 5.0, False, False], 'beta': [-2.0, -10.0, 10.0, False, False], 'Ep': [300.0, 10.0, 10000.0, False, False], 'norm': [1, 1e-10, 10000000000.0, False, False]}

(('alpha', '\\alpha', -1.0, -10.0, 5.0, False, False), ('beta', '\\beta', -2.0, -10.0, 10.0, False, False), ('Ep', '\\E_{\\rm p}', 300.0, 10.0, 10000.0, False, False), ('K', 'K', 1, 1e-10, 10000000000.0, False, False))

BlackBody
{'kT': [3.0, 0.0001, 200.0, False, False], 'norm': [1, 1e-10, 10000000000.0, False, False]}

(('kT', 'kT', 3.0, 0.0001, 200.0, False, False), ('K', 'K', 1.0, 1e-10, 10000000000